In [1]:
import sys
import csv
import math
import copy
import time
import numpy as np
from collections import Counter
from numpy import *
import pandas as pd
from random import shuffle

## Multiclass classifier decision tree using ID3 algorithm

In [2]:
feature = []

In [3]:
#normalize the entire dataset prior to learning using min-max normalization 
def normalize(matrix):
#   transfer the data metrix to np array in float type.
    a=np.array(matrix).astype(float)
#     print("normalizing the entire dataset:")
#     print(a)
#     print("Before normalizing")
#   apply the normalization along the 0 axis of a using the formula: (x - x_min)/(x_max - x_min)
    p1= a - a.min(axis=0)
    p2= a.max(axis=0)-a.min(axis=0)
    equation=p1/p2
    return equation

In [4]:
# reading from the file using numpy genfromtxt with delimiter ','
def load_csv(file):
    X=np.genfromtxt(file, delimiter=",",dtype=str)
    return (X)

In [5]:
#method to randomly shuffle the array using the numpy.random.shuffle()
def random_numpy_array(ar):
    np.random.shuffle(ar)
    return ar

In [6]:
#Normalize the data and generate the training labels,training features, test labels and test training
def generate_set(X):
#     print(X.shape[0])
#     store the label X[:,-1] to Y
    Y = X[:,-1]
#     reshape y to (Y's length, 1)
#     store it to j
    j = Y.reshape(len(Y),1)
    
#     print("J is",j)
#     create the new_X which exclude the label X[:,:-1]
    new_X = X[:,:-1]

#     normalize the data step
#     using our implemented function normalize()
    normalized_new_X = normalize(new_X)

#     add the label back to the normiazlied X
#     using np.concatenate along axis=1
    X = np.concatenate((normalized_new_X,j),axis=1)

#     store the size of rows of the normalized X with labels
    rows= X.shape[0]

#     use the 10% of the data to be the test set.
#     store the number of testing data
    testingSet = round(rows * 0.1)

#     set the starting idex to be 0
    start = 0

#     set the ending index to be the number of testing data
    end = testingSet

#     create a list that store all features of the testing data
    testingFeatures = []


#     create a list that store all labels of the testing data
    testingLabels = []

#     create a list that store all features of the training data
    trainingFeatures = []

#     create a list that store all labels of the training data
    trainingLabels = []

#     10-fold cross-validation:
    for i in range(10):
#         store the test set for corss-validation using X[start:end,:]
        testSet = X[start:end,:]

#         get training data before the testing data X[:start, :]
        trainBeforetest = X[:start,:]

#         get training data after the testing data X[:start, :]
        trainAftertest = X[end:,:]

#         form the new training set using np.concatenate
        trainSet = np.concatenate((trainBeforetest,trainAftertest),axis=0)

#         get the testing set labels
        testingSetLabels = testSet[:,-1]

#         flattent the labels
        testingSetLabels.flatten()

#         get the training set labels
        trainSetLabels = trainSet[:,-1]

#         flattent the labels
        trainSetLabels.flatten()

#         create the test set exclude the labels
        testSet = testSet[:,:-1].astype(np.float)

#         same for the training set
        trainSet = trainSet[:,:-1].astype(np.float)



#         append test data of this fold to the list
#         append test lables of this fold to the list

        testingFeatures.append(testSet)
        testingLabels.append(testingSetLabels)
#         do the same for the training set

        trainingFeatures.append(trainSet)
        trainingLabels.append(trainSetLabels)
#         update the index pointer

        start=end
        end=end+testingSet




#     return the fold list that contain data and label for both training and testing set.
    return trainingFeatures,trainingLabels,testingFeatures,testingLabels


In [7]:
#build a dictionary where the key is the class label and values are the features which belong to that class.
def build_dict_of_attributes_with_class_values(X,y):
#     init a dict for attributes
    attributedict = {}

#     init feature list.
    featurelist = []

#     for each feature in the dataset
    for i in range(X.shape[1]):
#         store the featur index
        featureindex = i
#     find all the value correspond to this feature
        values = X[:,i]
#     init an attribute list
        attributelist = []
#     init the counter to 0
        counter = 0
    # for each value in the "all the value correspond to this feature"
        for eachvalue in values:
            #             init a empty list that store the attribute value
            attributeValuesList = []
            #             append the this value to the list
            attributeValuesList.append(eachvalue)
            #             append the label of this value to the list
            attributeValuesList.append(y[counter])
            #             append this list to the attribute list.
            attributelist.append(attributeValuesList)
            #             increase the counter
            counter+=1
#         add this attribute list to the dict according to the feature index
        attributedict[featureindex] = attributelist
#         append the feature indx to the feature list.
        featurelist.append(featureindex)
#     return the dict and feature list.
    return attributedict,featurelist


In [8]:
# Iterative Dichotomiser 3 entropy calculation
def entropy(y):
#     init a class frequence dict
    classFreq = {}

#     init the attribute entropy to 0
    attributeEntropy = 0

#     for each label in y:
    for i in y:
#         this is label is already in the dict, we increase its feq
        if i in classFreq:
            classFreq[i]+=1
#          else, we set the freq to 1
        else:
            classFreq[i] = 1

#     calculate the cumulate entropy using the formula.
    for x in classFreq.values():
        p1= (-x/float(len(y)))
        p2= math.log(x/float(len(y)),2)
        attributeEntropy +=  p1*p2 

    return attributeEntropy


In [9]:
#Class node and explanation is self explaination
class Node(object):
#     init the node with val,lchild,rchild,thea and leaf.
    def __init__(self, val, lchild, rchild,thea,leaf):
        self.root_value = val
        self.root_left = lchild
        self.root_right = rchild
        self.theta = thea
        self.leaf = leaf

#     method to identify if the node is leaf
    def is_leaf(self):
        
        return self.leaf

#     method to return threshold value
    def ret_thetha(self):
        
        return self.theta
    
#     method return root value
    def ret_root_value(self):
        
        return self.root_value
    
#     method return left tree
    def ret_llist(self):
        
        return self.root_left

#     method return right tree
    def ret_rlist(self):
        
        return self.root_right

    def __repr__(self):
        return "(%r, %r, %r, %r)" %(self.root_value,self.root_left,self.root_right,self.theta)


In [10]:
#Decision tree object
class DecisionTree(object):
#     init a variable called fea_list
    fea_list = []

#     init the Dtree by setting the root node to None
    def __init__(self):
        self.root_node = None

    #method to return the major class value using Counter() and .most_common()
    def cal_major_class_values(self,class_values):
    
        return Counter(class_values).most_common(1)[0][0]

    #method to calculate best threshold value for each feature
    def cal_best_theta_value(self,ke,attri_list):
#         init a list for data
        dataList = []
#         init a list for class labes
        classLabels= []

#         for each attribute in the attri_list
        for i in attri_list:
#             append the data
            dataList.append(i[0])       
#             append the feature value.
            classLabels.append(i[1])

#         calculate the entropy of those feaure values
        entropyResult = entropy(classLabels)

#         init the max info gain = 0
        maxInfoGain = 0

#         init theta=0
        theta=0

#         init a list that store the best index on the left
        bestIndexLeft = []

#         init a list that store the best index on the right
        bestIndexRight = []

#         init a list that store class labels after split
        classLabelsAS = []

#         sort the data
        dataList.sort()

#         for each index of data:
        for i in range(len(dataList)-1):

#             calculate the current theta using data[i]+data[i+1])/ 2
            currentTheta = float(dataList[i]+dataList[i+1])/2
#             init a list that store index that less than theta
            indexLT = []
#             init a list that store value that less than theta
            valueLT = []
#             init a list that store index that greater than theta
            indexGT = []
#             init a list that store value that greater than theta
            valueGT =[]
#             init the counter to 0
            counter = 0
#             for each index and value in attri_list
            for c,j in enumerate(attri_list):
#                 if value less or equal than the current theta:
#                 update the "less" list of index and value
                if (j[0]<=currentTheta):
                    indexLT.append(c)
                    valueLT.append(j[1])
#                 else update the "greater" list of index and value
                else:
                    indexGT.append(c)
                    valueGT.append(j[1]) 
#           calculate the entropy of the "less" list
            entropyLess= entropy(valueLT)
#             calculate the entropy of the "greater" list
            entropyGreat= entropy(valueGT)
#             calculate the info gain using the formular.
            a= len(indexLT)/len(classLabels)
            b= len(indexGT)/len(classLabels)
            infoGain= a * entropyLess
            infoGain= infoGain+ (b*entropyGreat)
            infoGain = entropyResult -infoGain
#             if current info gain > max info gan
            if infoGain> maxInfoGain:
#                 update the info gain, 
                maxInfoGain=infoGain
#               the theta, the best index list of right, the best index list of left 
                theta= currentTheta
                bestIndexRight=indexGT
                bestIndexLeft=indexLT
#               class_labels_list_after_split
                classLabelsAS= valueGT+valueLT

#         return the max info gain, theata,the best left list,the best right list and class label after split
        return maxInfoGain, theta, bestIndexLeft,bestIndexRight, classLabelsAS

    #method to select the best feature out of all the features.
    def best_feature(self,dict_rep):
#         set key value to none
        keyValue=None

#         set best info gain to -1
        bestInfoGain=-1

#         set best theta to 0
        bestTheta=0

#         set best left list to empty
        bestLeftList=[]

#         set best right list to empty
        bestRightList=[]

#         set best class labels after split to empty
        bestClassLabelsAS=[]

#         init a result list
        resultList=[]

#         for each key in dict_rep:
        for ke in dict_rep.keys():
#             using cal_best_theta_value() and store all returned values
            maxInfoGain, theta, bestIndexLeft,bestIndexRight, classLabelsAS= self.cal_best_theta_value(ke, dict_rep[ke])
            

#             if info gian is greater than best info gain:
            if maxInfoGain> bestInfoGain:

#                 update info gain, theth, key value,
#                     left list, right list, class labels after split
                bestInfoGain=maxInfoGain
                bestTheta=theta
                keyValue= ke
                bestLeftList=bestIndexLeft
                bestRightList=bestIndexRight
                bestClassLabelsAS= classLabelsAS



    #         append the key value to the retrun list
        resultList.append(keyValue)
    #         append the theta value to the retrun list
        resultList.append(bestTheta)
    #         append the left list to the retrun list
        resultList.append(bestLeftList)
    #         append the right list to the retrun list
        resultList.append(bestRightList)
    #         append the class labels to the retrun list
        resultList.append(bestClassLabelsAS)
#         return the list.
        return resultList

    def get_remainder_dict(self,dict_of_everything,index_split):
        global fea_list
#         init a split dict
        split_dict={}

#         for each key "ke" in dict_of_everything:
        for ke in dict_of_everything.keys():
#             init a value list
            valueList=[]
#             init a modified list
            modifiedList=[]
#             get the corresponding values of the key"ke" 
            keValues= dict_of_everything[ke]
#             for each value and its corresponding index of the key"ke" 
            for eachvalue in range(len(keValues)):

#                 if index is not in the index_split:
                if eachvalue not in index_split:
#                     append it to the modified list and value list
                    modifiedList.append(keValues[eachvalue])
                    valueList.append(keValues[eachvalue][1])
#             add this modified list to the dict
            split_dict[ke]=modifiedList

#         return the splited dict and val list
        return split_dict, valueList

    #method to create decision tree
    def create_decision_tree(self, dict_of_everything,class_val,eta_min_val):
        global fea_list
        #if all the class labels are same, then we are set
        if len(set(class_val)) ==1:
#             print("Leaf node for set class is",class_val[0],len(class_val))
            n= Node(class_val[0],None,None,0,True)
            return n
        #if the no class vales are less than threshold, we assign the class with max values as the class label    
        elif len(class_val) < eta_min_val:
            classLabel = self.cal_major_class_values(class_val)
            m = Node(classLabel,None,None,0,True)
            return m
#         else:
        else:
#             using the best_feature to get best feature list
            bestFeatureList = self.best_feature(dict_of_everything)

#             store the node name, theta,left split, right split and class labes
            nodeName = bestFeatureList[0]
            theta = bestFeatureList[1]
            leftSplit = bestFeatureList[2]
            rightSplit = bestFeatureList[3]
            labels = bestFeatureList[4]

#             call get_remainder_dict to get left tree data
            leftSD, leftSC= self.get_remainder_dict(dict_of_everything, leftSplit)

#             call get_remainder_dict to get right tree data
            rightSD, rightSC= self.get_remainder_dict(dict_of_everything, rightSplit)

#             call create_decision_tree to get left tree based on the left tree data
            leftSN = self.create_decision_tree(leftSD,leftSC,eta_min_val)

#             call create_decision_tree to get right tree based on therightleft tree data
            rightSN = self.create_decision_tree(rightSD, rightSC, eta_min_val)

#             set the root node
            rn= Node(nodeName, rightSN, leftSN, theta, False)

#             return root node
            return rn
            
    #fit the decisin tree
    def fit(self, dict_of_everything,cl_val,features,eta_min_val):
#         set the fea_list the value of features
        global fea_list
        fea_list=features
#         set the root node using the function create_decision_tree()
        self.root_node= self.create_decision_tree(dict_of_everything,cl_val,eta_min_val)

        return self.root_node

    def classify(self,row,root):
#         init the test dict
        testDict={}
#         add row to the dict
        for k,j in enumerate(row): 
            testDict[k] = j
#         set the current node to root
        current_node = root
#         while the current node is not leaf:
        while not current_node.leaf:
#             implement the case whether the current shoud go to the left
            if testDict[current_node.root_value]<current_node.theta:
                current_node  = current_node.root_left
#             implement the case whether the current shoud go to the right
            else:
                current_node = current_node.root_right

        
#         return the calss of the current node
        return current_node.root_value
        
    #method to the labels for the test data
    def predict(self, X, root):
#         predict using the classify()
        PredictionList=[]
        for i in X:
            PredictionList.append(self.classify(i , root))
        return PredictionList

In [11]:
#calculating the predicited accuracy
def accuracy_for_predicted_values(test_class_names1,l):
#     init true and false count to 0
    falseCount=0
    trueCount=0
#     for each prediction,if predict is correct then, true++ else, false++
    for i in range(len(test_class_names1)):
        if test_class_names1[i] == l[i]:
            trueCount+=1
        else :
            falseCount+=1
#     print(trueCount)
#     print(trueCount+ falseCount)
#     print(len(test_class_names1))
    Accuracy= trueCount/(len(test_class_names1))    
#     return the acc
    return Accuracy

In [12]:
def main(num_arr, eta_min):
    eta_min_val = round(eta_min*num_arr.shape[0])
    #randomly shuffle the array so that we can divide the data into test/training
    randomArray = random_numpy_array(num_arr)
    #divide data into test labels,test features,training labels, training features
    trainingFeatures,trainingLabels, testingFeatures,testingLabels = generate_set(randomArray)
    
#     init cumulate acc to 0
    CumulateAccuracy = 0
    AccuracyList = []
    #ten fold iteration 
    for i in range(10):
        #build a dictionary with class labels and respective features values belonging to that class
        attributeDict,featureList = build_dict_of_attributes_with_class_values(trainingFeatures[i],trainingLabels[i])
        #instantiate decision tree instance
        Decisiontree = DecisionTree()
        # build the decision tree model.
        Decisiontree.fit(attributeDict,trainingLabels[i],featureList,eta_min_val)
        #predict the class labels for test features
        predictedClassLabels = Decisiontree.predict(testingFeatures[i],Decisiontree.root_node)
        #calculate the accuracy for the predicted values 
        accuracy = accuracy_for_predicted_values(predictedClassLabels,testingLabels[i])
        AccuracyList.append(accuracy)  
#         add acc to cumulate acc
        CumulateAccuracy+=accuracy

        print("Accuracy is ",accuracy*100, " %")
    print("Accuracy across 10-cross validation for",eta_min,"is",(float(CumulateAccuracy)/10)*100, " %")
    print("Standard dev for", eta_min, "is", np.std(AccuracyList))



In [13]:
eta_min_list = [0.05,0.10,0.15,0.20,0.25]
newfile = "iris.csv"
num_arr = load_csv(newfile)
for i in eta_min_list:
    main(num_arr,i)


Accuracy is  93.33333333333333  %
Accuracy is  93.33333333333333  %
Accuracy is  86.66666666666667  %
Accuracy is  100.0  %
Accuracy is  100.0  %
Accuracy is  93.33333333333333  %
Accuracy is  93.33333333333333  %
Accuracy is  100.0  %
Accuracy is  100.0  %
Accuracy is  100.0  %
Accuracy across 10-cross validation for 0.05 is 96.00000000000001  %
Standard dev for 0.05 is 0.044221663871405324
Accuracy is  100.0  %
Accuracy is  93.33333333333333  %
Accuracy is  93.33333333333333  %
Accuracy is  93.33333333333333  %
Accuracy is  86.66666666666667  %
Accuracy is  93.33333333333333  %
Accuracy is  100.0  %
Accuracy is  100.0  %
Accuracy is  100.0  %
Accuracy is  93.33333333333333  %
Accuracy across 10-cross validation for 0.1 is 95.33333333333334  %
Standard dev for 0.1 is 0.04268749491621898
Accuracy is  93.33333333333333  %
Accuracy is  100.0  %
Accuracy is  100.0  %
Accuracy is  100.0  %
Accuracy is  93.33333333333333  %
Accuracy is  80.0  %
Accuracy is  86.66666666666667  %
Accuracy is 

In [14]:
%%time
eta_min_list = [0.05,0.10,0.15,0.20,0.25]
newfile = "spambase.csv"
num_arr = load_csv(newfile)
for i in eta_min_list:
    main(num_arr,i)

Accuracy is  85.21739130434783  %
Accuracy is  81.52173913043478  %
Accuracy is  66.08695652173913  %
Accuracy is  47.391304347826086  %
Accuracy is  85.86956521739131  %
Accuracy is  43.26086956521739  %
Accuracy is  53.2608695652174  %
Accuracy is  74.1304347826087  %
Accuracy is  56.52173913043478  %
Accuracy is  83.69565217391305  %
Accuracy across 10-cross validation for 0.05 is 67.69565217391305  %
Standard dev for 0.05 is 0.15719420760190778
Accuracy is  87.60869565217392  %
Accuracy is  42.608695652173914  %
Accuracy is  91.08695652173913  %
Accuracy is  38.04347826086957  %
Accuracy is  84.1304347826087  %
Accuracy is  43.47826086956522  %
Accuracy is  88.47826086956522  %
Accuracy is  89.78260869565217  %
Accuracy is  82.82608695652173  %
Accuracy is  66.95652173913044  %
Accuracy across 10-cross validation for 0.1 is 71.50000000000001  %
Standard dev for 0.1 is 0.20768653218775143
Accuracy is  86.08695652173914  %
Accuracy is  41.95652173913044  %
Accuracy is  84.56521739130